In [1]:
import pandas as pd
import openrouteservice as ors
import folium
from PIL import Image
import numpy as np

import sys
sys.path.append("..")
import seaborn as sns
import matplotlib.pyplot as plt
from citibikeAPI import *
from utils import *
import time 

# dont need the below if we use local instance of ORS
# client = ors.Client(key='5b3ce3597851110001cf62487d33115ea15e483bb81a0b4cfcc899cb')

In [2]:
trip_data = '../../raw/202303-citibike-tripdata.csv'
topten = getTopTenConnections(trip_data)


c:\Users\lakeh\OneDrive\Documents\art\projects\citibike_data\TD-Citibike-Data-Viz\data\code\notebooks\..\citibikeAPI.py:136: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  trip_data = pd.read_csv(csv)


In [3]:
routeData = pd.read_csv('../../clean/routeData.csv').set_index('Unnamed: 0')

In [4]:

delay = 5
#initialize with an empty funtion call 
nDF, chgs = getEbikeEventsJSON()
oDF = nDF #set our old dataframe to our baseline from initialization 
print('sucessfully initialized')
time.sleep(delay) #wait after we setup
#this should run each minute 
starttime = time.time()
iter = 0

nDF , chgs = getEbikeEventsJSON(oDF)
oDF = nDF


#needs to output a json like this

newRoutes = {}
newRoutes['routes'] = []
for k,v in chgs.items():
    if v['gain_loss'] > 0: #if it is a route start
      try:
        tt = topten[k]
        weights = tt/tt.values.sum()
        for r in tt.index: 
          event = {}
          key = k + '-' + r
          event['id'] = routeData.loc[key, 'id']
          # event['timestamp'] = v['timestamp']
          event['weight'] = weights.loc[r]
          event['pts'] = routeData.loc[key, 'pts']
          event['offset'] = routeData.loc[key, 'offset']
          event['duration'] = 1 #TODO: improve this
          newRoutes['routes'].append(event)
      except: 
         pass 


sucessfully initialized


In [5]:
chgs.items()

dict_items([('Ditmars Blvd & 19 St', {'new_bike_count': 2, 'timestamp': 1682770827, 'gain_loss': -1})])

In [6]:
pd.DataFrame(newRoutes).to_json('../../clean/sampleRealTime.json')

In [7]:
pd.DataFrame(newRoutes['routes']).to_csv('../../clean/sampleRealTime.csv')

In [9]:
def getEbikeData(old_dataframe=None):

	changes = []

	station_status_url = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'
	#get station status
	with urllib.request.urlopen(station_status_url) as url:
		station_status_data = json.load(url)

	station_status_df = pd.DataFrame(station_status_data['data']['stations']).set_index('station_id')
	station_status_df = station_status_df[['num_ebikes_available', 'last_reported']] #extract cols of interest, here just ebike and update timestampe 
	#TODO: we should probably screen out ebike updates due to station-level events such as activation / nonactivation, rebalancing, etc 

	updated_df = station_status_df

	if old_dataframe is None: return station_status_df.sort_index(), pd.DataFrame(changes, columns = ['station_id', 'num_ebikes_available', 'gain_loss', 'window_start', 'window_end']).set_index('station_id')
	else: 
		udc = updated_df.sort_index()
		sdc = old_dataframe.sort_index()

		for i in sdc.index:
			if udc.loc[i,'num_ebikes_available'] == sdc.loc[i,'num_ebikes_available']: pass 
			else: 
				new_bike_count = udc.loc[i].to_list()[0]
				window_end = udc.loc[i].to_list()[1]
				old_bike_count = sdc.loc[i].to_list()[0]
				window_start = sdc.loc[i].to_list()[1]
				changes += [[i, new_bike_count, old_bike_count-new_bike_count, window_start, window_end]]
				# print('change!', udc.loc[i,'num_ebikes_available'], sdc.loc[i,'num_ebikes_available'], i, udc.loc[i, 'last_reported'])
		
		return udc, pd.DataFrame(changes, columns = ['station_id', 'num_ebikes_available', 'gain_loss', 'window_start', 'window_end']).set_index('station_id')
		# return new df, changes